### Download high-resolution images from wikiArt

In [17]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

In [18]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [19]:
def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    urls = []
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            img_url = img_url.split('!', 1)[0]
            urls.append(img_url)
    return urls

In [20]:
def get_all_addresses(addresses):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(addresses).content, "html.parser")
    urls = []
    
    for img in tqdm(soup.findAll("li", {"class": "painting-list-text-row"}), "Extracting images"):
        img_url = img.a.get('href')
        img_url = img_url.split('/', 2)[2]
        img_url = "https://uploads0.wikiart.org/images/" + img_url + ".jpg"
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(addresses, img_url)
        # remove URLs like '/hsts-pixel.gif?c=3.2.5'
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            img_url = img_url.split('!', 1)[0]
            urls.append(img_url)
    return urls

In [21]:
def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)

    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))

    # get the file name
    prefix = url.split('/', 5)[4]
    prefix = prefix.split('.', 1)[0]
    filename = os.path.join(pathname, prefix + "_" + url.split("/")[-1])

    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), "Downloading " + filename, total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))


In [22]:
def main(url, path):
    # get all images
    imgs = get_all_addresses(url)
    for img in imgs:
        # for each img, download it
        download(img, path)

### Download the files

In [23]:
artists_list = open("wikiart_artists.txt", "r")
lines = artists_list.readlines()

for url_addr in range(len(lines)):
    print(lines[url_addr].rstrip("\n")  + "/all-works/text-list")
    main(lines[url_addr].rstrip("\n")  + "/all-works/text-list", "wikiart_dataset")
    
#main("https://www.wikiart.org/en/tsuruko-yamazaki/", "wikiart_dataset")

https://www.wikiart.org/en/ad-reinhardt/all-works/text-list


Extracting images: 100%|████████████████████| 52/52 [00:00<00:00, 10393.81it/s]


https://www.wikiart.org/en/alekos-kontopoulos/all-works/text-list


Extracting images: 100%|█████████████████████| 79/79 [00:00<00:00, 7178.92it/s]


https://www.wikiart.org/en/alexander-liberman/all-works/text-list


...

https://www.wikiart.org/en/barnett-newman/all-works/text-list


Extracting images: 100%|████████████████████| 86/86 [00:00<00:00, 10741.81it/s]


https://www.wikiart.org/en/burgoyne-diller/all-works/text-list


Extracting images:   0%|                                | 0/26 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'get'

### Sort the .txt file alphabetically

In [8]:
txt = "wikiart_artists.txt"

f = open(txt , "r")
lines = f.readlines()
f.close()
lines.sort()

f = open(txt , "w")
for line in lines:
    f.write(line)
    
f.flush()
f.close()